<a href="https://colab.research.google.com/github/RaghavendraGaleppa/TravelAgentChatBot/blob/master/NewResnetAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [0]:
class BaseBlock(nn.Module):
  expansion = 1
  
  def __init__(self, input_planes, output_planes, stride=1, dim_change=None):
    super(BaseBlock, self).__init__()
    
    # Convolution Layers for the Base Block
    self.conv1 = nn.Conv2d(input_planes, output_planes, stride=stride, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(output_planes, output_planes, stride=1, kernel_size=3, padding=1)
    self.bn = nn.BatchNorm2d(output_planes)
    self.dim_change = dim_change
    
  def forward(self,x):
    # Residue
    res = x
    output = F.relu(self.bn(self.conv1(x)))
    output = self.bn(self.conv2(x))
    
    if self.dim_change is not None:
      res = self.dim_change(x)

    output += res
    output = F.relu(output)
    
    return output
    

In [0]:
class Bottleneck(nn.Module):
  expansion = 4
  
  def __init__(self, input_planes, output_planes, stride=1, dim_change=None):
    super(Bottleneck,self).__init__()
    
    self.conv1 = nn.Conv2d(input_planes, output_planes, stride=1, kernel_size=1)
    self.conv2 = nn.Conv2d(output_planes, output_planes, kernel_size=3, stride=stride,padding=1)
    self.conv3 = nn.Conv2d(output_planes, output_planes*self.expansion, kernel_size=1)
    self.bn1 = nn.BatchNorm2d(output_planes)
    self.bn2 = nn.BatchNorm2d(output_planes*self.expansion)
    self.dim_change = dim_change    
    
  def forward(self, x):
    res = x
    
    output = F.relu(self.bn1(self.conv1(x)))
    output = F.relu(self.bn1(self.conv2(output)))
    output = self.bn2(self.conv3(output))
    
    if self.dim_change is not None:
      res = self.dim_change(res)
      
    output += res
    output = F.relu(output)
    
    return output
    

In [0]:
class ResNet(nn.Module):
  
  def __init__(self, block, num_layers, classes=10):
    super(ResNet, self).__init__()
    
    self.input_planes = 64
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._layer(block,64, num_layers[0], stride=1)
    self.layer2 = self._layer(block,128,num_layers[1],stride=2)
    self.layer3 = self._layer(block,256, num_layers[2], stride=2)
    self.layer4 = self._layer(block,512, num_layers[3], stride=2)
    
    self.averagePool = nn.AvgPool2d(kernel_size=4, stride=1)
    
    self.fc = nn.Linear(512*block.expansion,classes)
    
  def _layer(self, block, planes, num_layers, stride=1):
    dim_change=None
    if stride != 1 or planes != self.input_planes*block.expansion:
      dim_change = nn.Sequential(nn.Conv2d(self.input_planes, planes*block.expansion,kernel_size=1, stride=stride),
                                nn.BatchNorm2d(planes*block.expansion))
    
    netLayers = []
    netLayers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
    self.input_planes = planes*block.expansion
    
    for i in range(1,num_layers):
      netLayers.append(block(self.input_planes,planes))
      self.input_planes = planes*block.expansion
      
    return nn.Sequential(*netLayers)
  
  def forward(self, x):
    x = F.relu(self.bn1(self.conv1(x)))
    
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    
    x = self.averagePool(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)
    
    return x

In [0]:
def test():
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
  ])
  
  train = torchvision.datasets.CIFAR10(root='./.data',train=True,download=True,transform=transform)
  trainset = torch.utils.data.DataLoader(train,batch_size=256,shuffle=True)
  
  test = torchvision.datasets.CIFAR10(root='./.data', train=False, download=True, transform=transform)
  testset = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False)
  
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  print(device)
  
  # Resnet 50
  net = ResNet(Bottleneck,[3,4,6,3])
  net.to(device)
  
  costFunc = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(net.parameters(), lr=0.02,momentum=0.9)
  
  for epoch in range(100):
    closs = 0
    for i, batch in enumerate(trainset,0):
      data, output = batch
      data, output = data.to(device), output.to(device)
      
      predic = net(data)
      loss = costFunc(predic, output)
      closs = loss.item()
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      if i%100 == 0:
        print('[%d %d] loss: %.4f' % (epoch+1, i+1, closs/1000))
        
    correctHits = 0
    totalHits = 0
      
    for batches in testset:
      data,output = batches
      data, output = data.to(device), output.to(device)
      
      pred = net(data)
      _,prediction = torch.max(pred.data, 1)
      totalHits = output.size(0)
      
      correctHits += (prediction==output).sum().item()
    print('Accuracy = '+str((correctHits/totalHits)*100))

In [0]:
test()

Files already downloaded and verified
Files already downloaded and verified
cuda:0
[1 1] loss: 0.0024
[1 101] loss: 0.0015
Accuracy = 32650.0
[2 1] loss: 0.0012
[2 101] loss: 0.0011
Accuracy = 39875.0
[3 1] loss: 0.0008
[3 101] loss: 0.0009
Accuracy = 43131.25
[4 1] loss: 0.0007
[4 101] loss: 0.0006
Accuracy = 47850.0
[5 1] loss: 0.0005
[5 101] loss: 0.0004
Accuracy = 48118.75
[6 1] loss: 0.0003
[6 101] loss: 0.0003
